Definition of extreme event according to Regulation EU 2017/1938 Article 6: 

"Extreme temperature duing a 7-day peak period with a statistical probability of one in 20 years"

Focus on the latter half i.e., 2000-2019

In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
# quantile equivalent of 7-day period once in 10 years (relaxed from 20 to include the second coldest)
q=7/365.25/10
print(q)

0.0019164955509924707


In [3]:
cwd=os.getcwd()
path_adjusted_temp=os.path.join(cwd, 'adjusted_ninja_temperature/')
all_csvs = glob.glob(path_adjusted_temp + "/*.csv")

countries=[filename[-6:-4] for filename in all_csvs]
print(countries)

['AT', 'BE', 'CZ', 'DE', 'DK', 'EE', 'GB', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LV', 'NL', 'PL', 'RO', 'SI', 'SK', 'CH', 'FI', 'NO', 'SE', 'PT', 'BG', 'ES', 'FR', 'LU', 'UA']


In [4]:
df_extreme={}

for country in countries:
    
    df_temp=pd.read_csv('adjusted_ninja_temperature/adjusted_ninja_temp_%s.csv' %country)
    df_temp['time']=pd.to_datetime(df_temp['time'])
    df_temp=df_temp.set_index('time')
    df_temp=df_temp.loc[df_temp.index.year>=2000] # consider only the second half of the data
    
    df_daily=df_temp.groupby(df_temp.index.date).mean()
    # take daily mean
    v_daily=df_daily.values
    # take 7 days average

    v_seven_days_average=np.array([
                v_daily[i:i+7].mean() for i in range(len(v_daily))]) # last 7 days are not 7-day averages but 
                                                                     # averages over the remaining days
    threshold=np.quantile(v_seven_days_average, q)        
    dates=df_daily.index[v_seven_days_average<threshold]
    averages=v_seven_days_average[v_seven_days_average<threshold]
    
    _df_extreme=pd.DataFrame(index=dates, columns=['7day_average'], data=averages)
    _df_extreme.index=pd.to_datetime(_df_extreme.index)
    _df_extreme.index.name='date'
    df_extreme[country]=_df_extreme
    
    print(country, threshold)
    
    for year, _df in _df_extreme.groupby(_df_extreme.index.year):
        for month, _df1 in _df.groupby(_df.index.month):
            print(_df1['7day_average'].idxmin(), _df1['7day_average'].min())

    

AT -8.293932747690079
2006-01-22 00:00:00 -9.606078800511304
2012-01-31 00:00:00 -10.270367550976307
2012-02-02 00:00:00 -10.96939939101035
2018-02-24 00:00:00 -8.378701796108675
BE -3.1107365978317585
2003-01-06 00:00:00 -3.402684221302238
2009-01-05 00:00:00 -3.617044008075279
2012-01-31 00:00:00 -4.543817593498688
2012-02-02 00:00:00 -5.422928506340996
CZ -10.299522481852435
2006-01-22 00:00:00 -13.23953096769666
2012-01-31 00:00:00 -12.801170570880656
2012-02-01 00:00:00 -13.016691124928387
DE -5.3883879004371815
2003-01-06 00:00:00 -5.84146515945259
2010-01-21 00:00:00 -5.493676292786094
2012-01-31 00:00:00 -7.644111533123505
2012-02-01 00:00:00 -8.340725726900375
DK -4.859088698967162
2003-01-04 00:00:00 -4.902508231962351
2010-12-21 00:00:00 -5.708893658890742
2012-01-31 00:00:00 -5.762324732891639
2012-02-02 00:00:00 -6.6618125659770895
EE -16.367814899943458
2006-01-17 00:00:00 -17.169339444458036
2010-01-21 00:00:00 -16.609344138863094
2011-02-15 00:00:00 -17.52598739067706
2

In [5]:
# record the lowest two years
df_peak_years=pd.DataFrame(index=countries, columns=['coldest_year', 'coldest_week_commencing',
                                                     'coldest_week_mean_temp', '2ndcoldest_year', 
                                                     '2ndcoldest_week_commencing', '2ndcoldest_week_mean_temp'])
for country in countries:
    
    _df=df_extreme[country]
    
    coldest_average=_df['7day_average'].min()
    coldest_day=_df['7day_average'].idxmin()
        
    
    _df=_df.loc[(_df.index<coldest_day-pd.Timedelta('30days')) | (_df.index>coldest_day+pd.Timedelta('30days'))]
    
    coldest2_average=_df['7day_average'].min()
    coldest2_day=_df['7day_average'].idxmin()
    
    df_peak_years.loc[country]=[coldest_day.year, coldest_day, coldest_average,
                               coldest2_day.year, coldest2_day, coldest2_average]
df_peak_years

,coldest_year,coldest_week_commencing,coldest_week_mean_temp,2ndcoldest_year,2ndcoldest_week_commencing,2ndcoldest_week_mean_temp
AT,2012,2012-02-02 00:00:00,-10.969399,2006,2006-01-22 00:00:00,-9.606079
BE,2012,2012-02-02 00:00:00,-5.422929,2009,2009-01-05 00:00:00,-3.617044
CZ,2006,2006-01-22 00:00:00,-13.239531,2012,2012-02-01 00:00:00,-13.016691
DE,2012,2012-02-01 00:00:00,-8.340726,2003,2003-01-06 00:00:00,-5.841465
DK,2012,2012-02-02 00:00:00,-6.661813,2010,2010-12-21 00:00:00,-5.708894
EE,2011,2011-02-15 00:00:00,-17.525987,2012,2012-01-30 00:00:00,-17.178925
GB,2010,2010-12-17 00:00:00,-1.215006,2010,2010-01-03 00:00:00,-1.135097
GR,2002,2002-01-02 00:00:00,0.923749,2006,2006-01-23 00:00:00,1.710421
HR,2012,2012-02-04 00:00:00,-8.205949,2003,2003-01-07 00:00:00,-6.264549
HU,2006,2006-01-22 00:00:00,-8.711588,2012,2012-02-02 00:00:00,-8.273053


In [6]:
# save
df_peak_years.to_csv('peak_years_00to19.csv')